In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


In [6]:
df = pd.read_excel('train.xlsx')

In [7]:
df['Number_of_Weeks_Used'].fillna(df.Number_of_Weeks_Used.median(), inplace=True)

In [8]:
df2 = pd.get_dummies(df, columns=['Crop', 'Soil'])

In [9]:
ext = df2[df2.Crop_status == 2]
df3 = pd.concat([df2,ext,ext,ext,ext])
df3.Crop_status.value_counts()

0    66716
1    11183
2    10505
Name: Crop_status, dtype: int64

In [10]:
ext = df2[df2.Crop_status == 2]
df3 = pd.concat([df2,ext,ext,ext,ext])
df3.Crop_status.value_counts()#dounsampling step
from sklearn.utils import resample

# Separating majority and minority classes
df_major = df3[df3.Crop_status==1]
df_minor= df3[df3.Crop_status==2]
 
# Downsample major class
df_majority_downsampled = resample(df_major,replace=False,n_samples=len(df_minor),random_state=42)
 
# Combining minor class with downsampled major class
dff = pd.concat([df_majority_downsampled, df_minor])
 
# Display new class counts
dff.Crop_status.value_counts(normalize = True)
dff.reset_index(inplace = True)
dff.drop(['index'], axis = 1, inplace = True)

#dounsampling step
from sklearn.utils import resample

# Separating majority and minority classes
df_major = df3[df3.Crop_status==0]
df_minor= dff[dff.Crop_status==2]
 
# Downsample major class
df_majority_downsampled = resample(df_major,replace=False,n_samples=len(df_minor),random_state=42)
 
# Combining minor class with downsampled major class
df4 = pd.concat([df_majority_downsampled,dff])
 
# Display new class counts
df4.Crop_status.value_counts(normalize = True)
df4.reset_index(inplace = True)
df4.drop(['index'], axis = 1, inplace = True)

In [11]:
df4.Crop_status.value_counts(normalize = True)

2    0.333333
1    0.333333
0    0.333333
Name: Crop_status, dtype: float64

In [12]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

In [13]:
X= df4.drop(['ID','Crop_status'],axis = 1)
y = df4['Crop_status']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 

In [16]:
st= StandardScaler()
X_train = st.fit_transform(X_train)
X_test  = st.transform(X_test)

In [17]:
from sklearn.ensemble import RandomForestClassifier 
forest = RandomForestClassifier(random_state = 42).fit(X_train, y_train)
forest.score(X_test,y_test)*100

73.85455006980581

In [18]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42).fit(X_train,y_train)
dt.score(X_test,y_test)*100

69.53928163472521

In [19]:
from sklearn.ensemble import VotingClassifier

In [20]:
vot = VotingClassifier(estimators=[('dt',dt),('Foresst',forest)],voting='hard').fit(X_train, y_train)
vot.score(X_test,y_test)*100

74.59068409696663